Testing to see if I can utilize the GPUs on my MacBook M1 Max.

Main test from here: https://github.com/mrdbourke/mac-ml-speed-test,
modified to run with the versions of the software I could install in this notebook.

In [69]:
import torch

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device = mps_device)
    print(x)
else:
    print("MPS device not found")

tensor([1.], device='mps:0')


In [70]:
import timeit

import random

x = torch.ones(500000000, device='mps')
timeit.timeit(lambda:x * random.randint(0, 100), number=1)

0.0003037499263882637

In [71]:
x = torch.ones(50000000, device="cpu")
timeit.timeit(lambda:x * random.randint(0,100), number = 1)

0.012535833055153489

There definitely is a time difference, but the source I am comparing this to had 0.0003982 and 0.0112392, so overall the numbers I am seeing are off by a bit.


In [84]:
from time import process_time
import torch

def testgpu():
    if torch.backends.mps.is_available():
        mps_device = torch.device("mps")
    t0 = process_time()
    x = torch.ones(n1, device=mps_device)
    y = x + torch.rand(n1, device=mps_device)
    t1 = process_time()
    print(f"Total time with gpu ({n1}): {t1-t0}")
    t0 = process_time()
    x = torch.ones(n2, device=mps_device)
    y = x + torch.rand(n2, device=mps_device)
    t1 = process_time()
    print(f"Total time with gpu ({n2}): {t1-t0}")

def testcpu():
    t0 = process_time()
    x = torch.ones(n1)
    y = x + torch.rand(n1)
    t1 = process_time()
    print(f"Total time with cpu ({n1}): {t1-t0}")
    t0 = process_time()
    x = torch.ones(n2)
    y = x + torch.rand(n2)
    t1 = process_time()
    print(f"Total time with cpu ({n2}): {t1-t0}")

if __name__ == '__main__':
    n1 = 10000
    n2 = 100000000
    for x in range(n1):
        testcpu()
    for x in range(n1):
        testgpu()

Total time with cpu (10000): 0.0004969999999957508
Total time with cpu (100000000): 0.44410700000025827
Total time with cpu (10000): 9.700000009615906e-05
Total time with cpu (100000000): 0.43829399999958696
Total time with cpu (10000): 0.0001229999998031417
Total time with cpu (100000000): 0.44238900000073045
Total time with cpu (10000): 9.099999988393392e-05
Total time with cpu (100000000): 0.4459919999999329
Total time with cpu (10000): 9.200000022246968e-05
Total time with cpu (100000000): 0.4349770000007993
Total time with cpu (10000): 8.999999954539817e-05
Total time with cpu (100000000): 0.43922099999963393
Total time with cpu (10000): 8.000000070751412e-05
Total time with cpu (100000000): 0.44181899999966845
Total time with cpu (10000): 9.799999952520011e-05
Total time with cpu (100000000): 0.43906199999946693
Total time with cpu (10000): 8.900000011635711e-05
Total time with cpu (100000000): 0.44361599999956525
Total time with cpu (10000): 8.499999967170879e-05
Total time with

KeyboardInterrupt: 

In [73]:
import argparse
import os
from pathlib import Path
from timeit import default_timer as timer

import torch
import torchvision
import torchvision.transforms.v2 as transforms # use v2 transforms for faster augmentations
import pandas as pd

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from tqdm.auto import tqdm



As this was a command line function, need some modifications to make it work in a notebook.
See: https://stackoverflow.com/questions/48796169/how-to-fix-ipykernel-launcher-py-error-unrecognized-arguments-in-jupyter

In [74]:
print(torchvision.__version__)

0.15.2a0


Latest version of torchvision is 0.17, but I can't convince conda to install that...
https://pytorch.org/vision/0.15/transforms.html#transforms-scriptability

In [75]:
    # Create DataLoaders
    def create_dataloaders(batch_size, num_workers=NUM_WORKERS):
        train_dataloader = DataLoader(train_data,
                                      batch_size=batch_size,
                                      shuffle=True,
                                      num_workers=num_workers,
                                      pin_memory=False) # note: if you pin memory, you may get "too many workers" errors when recreating DataLoaders, see: https://github.com/Lightning-AI/pytorch-lightning/issues/18487#issuecomment-1740244601

        test_dataloader = DataLoader(test_data,
                                     batch_size=batch_size,
                                     shuffle=False,
                                     num_workers=num_workers,
                                     pin_memory=False)

        return train_dataloader, test_dataloader

In [76]:

    ### Train Step ###
    def train_step(model: torch.nn.Module, 
                   dataloader: torch.utils.data.DataLoader, 
                   loss_fn: torch.nn.Module, 
                   optimizer: torch.optim.Optimizer,
                   device: torch.device):
        # Put model in train mode
        model.train()
        
        # Setup train loss and train accuracy values
        train_loss, train_acc = 0, 0
        
        # Loop through data loader data batches
        for batch, (X, y) in tqdm(enumerate(dataloader), total=len(dataloader)):
            # Send data to target device
            X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)
    #         X, y = X.to(device, non_blocking=True, memory_format=torch.channels_last), y.to(device, non_blocking=True)
    #         X, y = X.to(device), y.to(device)

            # 1. Forward pass
            y_pred = model(X)

            # 2. Calculate  and accumulate loss
            loss = loss_fn(y_pred, y)
            train_loss += loss.item() 

            # 3. Optimizer zero grad
            optimizer.zero_grad()

            # 4. Loss backward
            loss.backward()

            # 5. Optimizer step
            optimizer.step()

            # Calculate and accumulate accuracy metric across all batches
            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            train_acc += (y_pred_class == y).sum().item()/len(y_pred)
            
        # Adjust metrics to get average loss and accuracy per batch 
        train_loss = train_loss / len(dataloader)
        train_acc = train_acc / len(dataloader)
        return train_loss, train_acc

In [77]:

    ### Test Step ###
    def test_step(model: torch.nn.Module, 
                dataloader: torch.utils.data.DataLoader, 
                loss_fn: torch.nn.Module,
                device: torch.device):
        # Put model in eval mode
        model.eval() 
        
        # Setup test loss and test accuracy values
        test_loss, test_acc = 0, 0
        
        # Turn on inference context manager
        with torch.inference_mode():
            # Loop through DataLoader batches
            for batch, (X, y) in tqdm(enumerate(dataloader), total=len(dataloader)):
                # Send data to target device
                X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)
    #             X, y = X.to(device, non_blocking=True, memory_format=torch.channels_last), y.to(device, non_blocking=True)
    #             X, y = X.to(device), y.to(device)
        
                # 1. Forward pass
                test_pred_logits = model(X)

                # 2. Calculate and accumulate loss
                loss = loss_fn(test_pred_logits, y)
                test_loss += loss.item()
                
                # Calculate and accumulate accuracy
                test_pred_labels = test_pred_logits.argmax(dim=1)
                test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
                
        # Adjust metrics to get average loss and accuracy per batch 
        test_loss = test_loss / len(dataloader)
        test_acc = test_acc / len(dataloader)
        return test_loss, test_acc

In [78]:
    def train_and_time(batch_sizes=BATCH_SIZES,
                       epochs=EPOCHS,
                       device=device):

        batch_size_training_results = []

        for batch_size in batch_sizes:
            print(f"[INFO] Training with batch size {batch_size} for {epochs} epochs...")
            # Create an instance of resnet50
            model = torchvision.models.resnet50(num_classes=100).to(device)
            # model = torch.compile(model) # potential way to speed up model

            # Setup loss function and optimizer
            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

            # Create DataLoaders
            train_dataloader, test_dataloader = create_dataloaders(batch_size=batch_size)

            try:
                # Start the timer
                start_time = timer()

                # Train model
                model_results = train_and_test_model(model=model, 
                                                    train_dataloader=train_dataloader,
                                                    test_dataloader=test_dataloader,
                                                    optimizer=optimizer,
                                                    loss_fn=loss_fn, 
                                                    epochs=epochs,
                                                    device=device,
                                                    eval=False) # don't eval, just test training time

                # End the timer
                end_time = timer()

                total_training_time = end_time - start_time
                avg_time_per_epoch = total_training_time / epochs

                batch_size_training_results.append({"batch_size": batch_size,
                                                    "avg_time_per_epoch": avg_time_per_epoch})
                save_results(batch_size_training_results)
                print(f"[INFO] Finished training with batch size {batch_size} for {epochs} epochs, total time: {round(total_training_time, 3)} seconds, avg time per epoch: {round(avg_time_per_epoch, 3)} seconds\n\n")

            except Exception as e:
                print(f"[INFO] Error: {e}")
                print(f"[INFO] Failed training with batch size {batch_size} for {epochs} epochs...\n\n")
                batch_size_training_results.append({"batch_size": batch_size,
                                                    "avg_time_per_epoch": "FAILED"})
                save_results(batch_size_training_results)
                break
                
        return batch_size_training_results

In [79]:
    
    def save_results(batch_size_training_results, target_dir="results_pytorch_cv"):
        # Create CSV filename
        if GPU_NAME:
            csv_filename = f"{GPU_NAME.replace(' ', '_')}_{DATASET_NAME}_{MODEL_NAME}_{INPUT_SHAPE[-1]}_{BACKEND}_results.csv"
        else:
            csv_filename = f"{CPU_PROCESSOR}_{DATASET_NAME}_{MODEL_NAME}_{INPUT_SHAPE[-1]}_{BACKEND}_results.csv"

        # Make the target results directory if it doesn't exist (include the parents)
        target_results_dir = target_dir
        results_path = Path("results") / target_results_dir
        results_path.mkdir(parents=True, exist_ok=True)
        csv_filepath = results_path / csv_filename

        # Turn dict into DataFrame 
        df = pd.DataFrame(batch_size_training_results) 

        # Save to CSV
        print(f"[INFO] Saving results to: {csv_filepath}")
        df.to_csv(csv_filepath, index=False)

In [80]:
    # 1. Take in various parameters required for training and test steps
    def train_and_test_model(model: torch.nn.Module, 
                            train_dataloader: torch.utils.data.DataLoader, 
                            test_dataloader: torch.utils.data.DataLoader, 
                            optimizer: torch.optim.Optimizer,
                            loss_fn: torch.nn.Module,
                            epochs: int,
                            device: torch.device,
                            eval: bool=False):
        
        print(f"[INFO] Training model {model.__class__.__name__} on device '{device}' for {epochs} epochs...")
        
        results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

        # Loop through training and testing steps for a number of epochs
        for epoch in tqdm(range(epochs)):
            # Do eval before training (to see if there's any errors)
            if eval:
                test_loss, test_acc = test_step(model=model,
                                            dataloader=test_dataloader,
                                            loss_fn=loss_fn,
                                            device=device)
            
            train_loss, train_acc = train_step(model=model,
                                            dataloader=train_dataloader,
                                            loss_fn=loss_fn,
                                            optimizer=optimizer,
                                            device=device)
            
            
            # Print out what's happening
            print(
                f"Epoch: {epoch+1} | "
                f"train_loss: {train_loss:.4f} | "
                f"train_acc: {train_acc:.4f} | "
            )

            if eval:
                print(
                    f"Epoch: {epoch+1} | "
                    f"test_loss: {test_loss:.4f} | "
                    f"test_acc: {test_acc:.4f} | "
                )

            # Save results to dictionary
            results["train_loss"].append(train_loss)
            results["train_acc"].append(train_acc)
            if eval:
                results["test_loss"].append(test_loss)
                results["test_acc"].append(test_acc)
        
        return results


In [81]:

    CPU_PROCESSOR = None

    ### Get CPU Processor name ###
    if not CPU_PROCESSOR:
        try:
            import cpuinfo
            CPU_PROCESSOR = cpuinfo.get_cpu_info().get("brand_raw").replace(" ", "_")
            print(f"[INFO] CPU Processor: {CPU_PROCESSOR}")
        except Exception as e:
            print(f"Error: {e}, may have failed to get CPU_PROCESSOR name from cpuinfo, please install cpuinfo or set CPU_PROCESSOR manually") 

    ### Setup device ###
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print(f"[INFO] MPS device found, using device: {device}")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"[INFO] CUDA device found, using device: {device}")
    else:
        device = torch.device("cpu")
        print (f"[INFO] MPS or CUDA device not found, using device: {device} (results will be much slower than using MPS or CUDA)")

    # Prevent torch from erroring with too many files open (happens on M3)
    # See: https://github.com/pytorch/pytorch/issues/11201, https://github.com/CVMI-Lab/PLA/issues/20 
    torch.multiprocessing.set_sharing_strategy('file_system')

    # Set random seed
    torch.manual_seed(42)

    ### Set constants ###
    GPU_NAME = "mps"
    BACKEND = "pytorch"
    MODEL_NAME = "resnet50"
    IMAGE_SIZE = 32
    INPUT_SHAPE = (3, IMAGE_SIZE, IMAGE_SIZE)
    NUM_WORKERS = os.cpu_count()
    EPOCHS = 5
    BATCH_SIZES = [16, 32, 64, 128, 256, 512, 1024]
    DATASET_NAME = "CIFAR100"

    print(f"[INFO] Testing model: {MODEL_NAME} on {DATASET_NAME} dataset with input shape {INPUT_SHAPE} for {EPOCHS} epochs across batch sizes: {BATCH_SIZES}")


    ### Prepare Data ### 
    simple_transform = transforms.Compose([
        transforms.Resize(size=IMAGE_SIZE),
        # for torchvision 0.15 I believe ToImage must be ToTensor
        transforms.ToTensor(), 
        # again, since I seems stuck on 0.15, where there is no scale option
        #transforms.ToDtype(torch.float32, scale=True)
        transforms.ConvertDtype(torch.float32)

    ])

    # Get Datasets
    train_data = datasets.CIFAR10(root="data",
                                train=True,
                                transform=simple_transform,
                                download=True)

    test_data = datasets.CIFAR10(root="data",
                                train=False,
                                transform=simple_transform,
                                download=True)

    print(f"[INFO] Number of training samples: {len(train_data)}, number of testing samples: {len(test_data)}")

    ### Train an time model ### 
    batch_size_training_results = train_and_time(batch_sizes=BATCH_SIZES,
                                                 epochs=EPOCHS,
                                                 device=device)

    print("[INFO] Finished training with all batch sizes.")        

    print(f"[INFO] Results:\n{batch_size_training_results}")

[INFO] CPU Processor: Apple_M1_Max
[INFO] MPS device found, using device: mps
[INFO] Testing model: resnet50 on CIFAR100 dataset with input shape (3, 32, 32) for 5 epochs across batch sizes: [16, 32, 64, 128, 256, 512, 1024]


/Users/rff/anaconda3/envs/MacML/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified
[INFO] Number of training samples: 50000, number of testing samples: 10000
[INFO] Training with batch size 16 for 5 epochs...
[INFO] Training model ResNet on device 'mps' for 5 epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

/Users/rff/anaconda3/envs/MacML/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py:222: UserWarning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1712648268885/work/aten/src/ATen/ParallelNative.cpp:228.)
  torch.set_num_threads(1)
/Users/rff/anaconda3/envs/MacML/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py:222: UserWarning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1712648268885/work/aten/src/ATen/ParallelNative.cpp:228.)
  torch.set_num_threads(1)
/Users/rff/anaconda3/envs/MacML/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py:222: UserWarning: Cannot set number of intraop threads after parallel work has started 

  0%|          | 0/3125 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/rff/anaconda3/envs/MacML/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/rc/mdny9tkx4fzfdtr0kdhl5j540000gn/T/ipykernel_68969/1128608748.py", line 69, in <module>
    batch_size_training_results = train_and_time(batch_sizes=BATCH_SIZES,
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/rc/mdny9tkx4fzfdtr0kdhl5j540000gn/T/ipykernel_68969/4130627343.py", line 25, in train_and_time
    model_results = train_and_test_model(model=model,
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/rc/mdny9tkx4fzfdtr0kdhl5j540000gn/T/ipykernel_68969/2430635053.py", line 24, in train_and_test_model
    train_loss, train_acc = train_step(model=model,
                            ^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/rc/mdny9tkx4fzfdtr0kdhl5j540000gn/T/ipykernel_68969/3617

##Final Results##

These results of this test appear to be twice as slow, running on an M1 Pro, as the original source demonstrated:

[{'batch_size': 16, 'avg_time_per_epoch': 407.5454948831815}, {'batch_size': 32, 'avg_time_per_epoch': 233.77520572501234}, {'batch_size': 64, 'avg_time_per_epoch': 150.9472023168113}, {'batch_size': 128, 'avg_time_per_epoch': 109.9742384667974}, {'batch_size': 256, 'avg_time_per_epoch': 92.17897170837969}, {'batch_size': 512, 'avg_time_per_epoch': 91.76529929162935}, {'batch_size': 1024, 'avg_time_per_epoch': 91.80344411660917}]

https://github.com/mrdbourke/mac-ml-speed-test

In [ ]:
??transforms